<a href="https://colab.research.google.com/github/Nitya131003/Bitcoin/blob/main/Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


In [ ]:
!pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle datasets download -d mczielinski/bitcoin-historical-data

In [ ]:
!unzip bitcoin-historical-data.zip

In [ ]:
!ls

In [ ]:
import pandas as pd
df = pd.read_csv('btcusd_1-min_data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.describe

In [ ]:
df.info()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df['Close'])
plt.title('Bitcoin Close price.', fontsize=15)
plt.ylabel('Price in dollars.')
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['Volume'], axis=1)

In [ ]:
df.info()

In [ ]:
features = ['Open', 'High', 'Low', 'Close']

plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
  plt.subplot(2,2,i+1)
  sb.histplot(df[col], kde=True)
plt.show()

In [ ]:
# Convert the 'Timestamp' column to datetime objects directly from Unix timestamps
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')  # 'unit' set to 's' for seconds

# Split the datetime into separate date and time columns
df['date'] = df['Timestamp'].dt.date  # Extract date part
df['time'] = df['Timestamp'].dt.time  # Extract time part

df.head()


In [ ]:
df.groupby('time').size().plot(kind='barh', color=sb.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
data_grouped = df.groupby('date')[['Open', 'High', 'Low', 'Close']].mean()

for i, col in enumerate(['Open', 'High', 'Low', 'Close']):
  plt.subplot(2,2,i+1)
  data_grouped[col].plot.bar()
plt.show()

In [ ]:
df['open-close']  = df['Open'] - df['Close']
df['low-high']  = df['Low'] - df['High']
df['target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

In [ ]:
plt.pie(df['target'].value_counts().values,
        labels=[0, 1], autopct='%1.1f%%')
plt.show()

In [ ]:
df.info()

In [ ]:
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Calculate correlation on numeric columns only
corr_matrix = numeric_df.corr()

# Plot the heatmap with correlations
plt.figure(figsize=(10, 10))
sb.heatmap(corr_matrix > 0.9, annot=True, cbar=False, cmap="coolwarm")
plt.show()

In [ ]:
features = df[['open-close', 'low-high']]
target = df['target']

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_valid, Y_train, Y_valid = train_test_split(
    features, target, test_size=0.1, random_state=2022)
print(X_train.shape, X_valid.shape)


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Define models with optimized parameters
models = [
    LogisticRegression(),
   # SVC(kernel='rbf', probability=True),  # Changed kernel to 'rbf' for faster performance
    RandomForestClassifier(n_estimators=50, n_jobs=-1)
]

# Select 20% of data randomly for SVC to reduce training time
subset_indices = np.random.choice(len(X_train), size=int(0.2 * len(X_train)), replace=False)
X_train_small, Y_train_small = X_train[subset_indices], Y_train[subset_indices]

# Train and evaluate each model
for model in models:
    if isinstance(model, SVC):  # Use the smaller subset for SVC
        model.fit(X_train_small, Y_train_small)
    else:
        model.fit(X_train, Y_train)

    # Print model details and AUC-ROC scores
    print(f'{model} :')
    print('Training Accuracy:', metrics.roc_auc_score(Y_train, model.predict_proba(X_train)[:, 1]))
    print('Validation Accuracy:', metrics.roc_auc_score(Y_valid, model.predict_proba(X_valid)[:, 1]))
    print()


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(models[0], X_valid, Y_valid)
plt.show()